# Introduction

In [1]:
from uarray import *

## Creating Arrays

All arrayus 

### Scalar

We can add custom values as scalars:

In [2]:
a_scalar = scalar(123)
print(a_scalar)
a_scalar

Array(NoLengthAccessor(), 123)


Array(NoLengthAccessor(), ScalarAccessor('123'))

Now we can take the shape of a scalar:

In [3]:
a_scalar_shape = Shape(a_scalar)
a_scalar_shape

Shape(Array(NoLengthAccessor(), ScalarAccessor('123')))

We see that the shape is not computed.

It just contains a reference to the arrays it operates on:

In [4]:
a_scalar_shape.operands

[Array(NoLengthAccessor(), ScalarAccessor('123'))]

It also has a shorter representation we can print based on the Mathematics of Arrays symbols:

In [5]:
print(a_scalar_shape)

ρ(Array(NoLengthAccessor(), 123))


We call `replace` to keep replacing expressions until no more are matched:

In [6]:
replace_debug(a_scalar_shape, False)

ρ(Array(NoLengthAccessor(), 123))

  Shape(Array(NoLengthAccessor(), ScalarAccessor('123')))
->ShapeInner(ScalarAccessor('()'), Array(NoLengthAccessor(), ScalarAccessor('123')))

ShapeInner((), Array(NoLengthAccessor(), 123))

  ShapeInner(ScalarAccessor('()'), Array(NoLengthAccessor(), ScalarAccessor('123')))
->Array(ScalarAccessor('0'), VectorAccessor('()'))

Array(0, <>)


Array(ScalarAccessor('0'), VectorAccessor('()'))

### Vector

We see that the shape of a scalar is `Vector`. `Vector` is a special operation that takes any number of operands. Each operand should be a scalar. We can create them with the `vector` helper function:

In [7]:
??vector

Signature: vector(*values)
Docstring: <no docstring>
Source:   
def vector(*values):
    accessor = VectorAccessor(tuple(ScalarAccessor(v) for v in values))
    return Array(ScalarAccessor(len(values)), accessor)
File:      ~/p/uarray/uarray/uarray.py
Type:      function


In [8]:
a_vector = vector(1, 2, 3)
print(a_vector)
a_vector

Array(3, <1 2 3>)


Array(ScalarAccessor('3'), VectorAccessor("(ScalarAccessor('1'), ScalarAccessor('2'), ScalarAccessor('3'))"))

We can see the shape of a vector:

In [9]:
a_vector_shape = replace(Shape(a_vector))
print(a_vector_shape)

Array(1, <3>)


Which is itself a vector. 

### Iota

Now let's do something more interesting and create a range of values:

In [10]:
a_range = Iota(scalar(10))
print(a_range)

ι(Array(NoLengthAccessor(), 10))


In [11]:
print(replace(Shape(a_range)))

Array(1, <10>)


This is a vector of length 10. Let's see that the 5th element is the integer 5.

In [12]:
print(replace(Index(vector(5), a_range)))

Array(NoLengthAccessor(), 5)


Here we use a new command `Index` which takes in a vector of indices and the an array and returns a subarray.

Let's try adding taking it's sum:

In [13]:
replace(ReduceVector(scalar(0), ScalarAccessor(Add), a_range))

Array(NoLengthAccessor(), ScalarAccessor('45'))

In [14]:
sum(range(10))

45

# Paper Example

Now let's look at the example from the paper in `uarray-docs`.

In [15]:
Z = AbstractWithDimension(0, "Z")
A = AbstractWithDimension(1, "A")
B = AbstractWithDimension(2, "B")
pprint.pprint(replace(Z))
replaced_A = replace(Get(UnboundAccessor("i_0"), Content(A)))
pprint.pprint(replaced_A)
pprint.pprint(replace(Get(UnboundAccessor("i_0"), Content(B))))
pprint.pprint(replace(Get(UnboundAccessor("i_1"), Content(Get(UnboundAccessor("i_0"), Content(B))))))


A_value = Iota(scalar(10))
A_subs = matchpy.substitute(replaced_A, {"A_content": Content(A_value), "A_shape": Shape(A_value)})
pprint.pprint(replace(A_subs))

Array(NoLengthAccessor(), UnboundAccessor('', variable_name=Z_content))
Array(NoLengthAccessor(),
      Content(Get(UnboundAccessor('', variable_name=i_0),
                  UnboundAccessor('', variable_name=A_content))))
Array(Content(Get(ScalarAccessor('1'),
                  UnboundAccessor('', variable_name=B_shape))),
      GetBySubstituting(ScalarAccessor("'idx_5'"),
                        Array(NoLengthAccessor(),
                              Content(Get(UnboundAccessor('', variable_name=idx_5),
                                          Content(Get(UnboundAccessor('', variable_name=i_0),
                                                      UnboundAccessor('', variable_name=B_content))))))))
Array(NoLengthAccessor(),
      Content(Get(UnboundAccessor('', variable_name=i_1),
                  Content(Get(UnboundAccessor('', variable_name=i_0),
                              UnboundAccessor('', variable_name=B_content))))))
Array(NoLengthAccessor(), UnboundAccessor('', variable_n

In [16]:
A = AbstractWithDimension(3, "A")
B = AbstractWithDimension(3, "B")
r = InnerProduct(
    Index(vector(1, 0), A),
    ScalarAccessor(Add),
    ScalarAccessor(Multiply),
    Index(
        vector(2),
        OuterProduct(
            Index(vector(1, 0), A),
            ScalarAccessor(Multiply),
            Index(vector(0, 1), B)
        )
    )
)
print(r)

((Array(2, <1 0>) ψ A^3) +·* (Array(1, <2>) ψ ((Array(2, <1 0>) ψ A^3) ·* (Array(2, <0 1>) ψ B^3))))


In [24]:
replaced_r = replace(r)
print(replaced_r)

(Array(Content(Get(2, UnboundAccessor(A_shape))), GetBySubstituting(idx_44, Array(NoLengthAccessor(), Content(Get(UnboundAccessor(idx_44), Content(Get(0, Content(Get(1, UnboundAccessor(A_content)))))))))) +·* Array(Content(Get(2, UnboundAccessor(B_shape))), GetBySubstituting(idx_52, (Array(NoLengthAccessor(), Content(Get(2, Content(Get(0, Content(Get(1, UnboundAccessor(A_content)))))))) * Array(NoLengthAccessor(), Content(Get(UnboundAccessor(idx_52), Content(Get(1, Content(Get(0, UnboundAccessor(B_content))))))))))))


In [25]:
pprint.pprint(replaced_r)

InnerProduct(Array(Content(Get(ScalarAccessor('2'),
                               UnboundAccessor('', variable_name=A_shape))),
                   GetBySubstituting(ScalarAccessor("'idx_44'"),
                                     Array(NoLengthAccessor(),
                                           Content(Get(UnboundAccessor('', variable_name=idx_44),
                                                       Content(Get(ScalarAccessor('0'),
                                                                   Content(Get(ScalarAccessor('1'),
                                                                               UnboundAccessor('', variable_name=A_content)))))))))),
             ScalarAccessor("Add['+', Arity(min_count=2, fixed_size=True), infix]"),
             ScalarAccessor("Multiply['*', Arity(min_count=2, fixed_size=True), infix]"),
             Array(Content(Get(ScalarAccessor('2'),
                               UnboundAccessor('', variable_name=B_shape))),
                   

OK this is a little bit much. It's pretty hard to understand this and if it is optimized. Let's say we do know the Shape information for A and B and put that in.

In [27]:
with_shapes = replace(matchpy.substitute(replaced_r, {
    "A_shape": Content(vector(2, 3, 5)),
    "B_shape": Content(vector(2, 3, 5)),
}))

In [29]:
pprint.pprint(with_shapes)

InnerProduct(Array(ScalarAccessor('5'),
                   GetBySubstituting(ScalarAccessor("'idx_44'"),
                                     Array(NoLengthAccessor(),
                                           Content(Get(UnboundAccessor('', variable_name=idx_44),
                                                       Content(Get(ScalarAccessor('0'),
                                                                   Content(Get(ScalarAccessor('1'),
                                                                               UnboundAccessor('', variable_name=A_content)))))))))),
             ScalarAccessor("Add['+', Arity(min_count=2, fixed_size=True), infix]"),
             ScalarAccessor("Multiply['*', Arity(min_count=2, fixed_size=True), infix]"),
             Array(ScalarAccessor('5'),
                   GetBySubstituting(ScalarAccessor("'idx_52'"),
                                     Multiply(Array(NoLengthAccessor(),
                                                    Content(